In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [16]:
import pandas as pd
import numpy as np

import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)
pl.Config.set_tbl_cols(-1)

polars.config.Config

In [21]:
# Lee la base de GRD del Torax
df_torax = (
    pl.scan_csv("../data/processed/df_procesada.csv", separator=";", infer_schema_length=0)
    .filter((pl.col("COD_HOSPITAL") == "112103") & (pl.col("TIPO_ACTIVIDAD") == "HOSPITALIZACIÓN"))
    .cast({"ESTANCIA": pl.Int32})
    .collect()
)

In [22]:
# Codigos de Trasplante pulmon unilateral, bilateral y cardiaco, respectivamente
CODIGOS_TRASPLANTE = ["33.51", "33.52", "37.51"]

In [23]:
df_trasplantes = df_torax.filter(pl.col("PROCEDIMIENTO1").is_in(CODIGOS_TRASPLANTE))

In [34]:
df_trasplantes.group_by(["ANIO_EGRESO", "PROCEDIMIENTO1"]).agg(
    [
        pl.col("ESTANCIA").sum().alias("dias_estada_totales"),
        pl.len().alias("n_egresos"),
        pl.col("ESTANCIA").mean().alias("dias_estada_promedio"),
    ]
).sort(["ANIO_EGRESO", "PROCEDIMIENTO1", "dias_estada_promedio"])

ANIO_EGRESO,PROCEDIMIENTO1,dias_estada_totales,n_egresos,dias_estada_promedio
str,str,i32,u32,f64
"""2019""","""33.52""",504,7,72.0
"""2019""","""37.51""",812,12,67.666667
"""2020""","""33.52""",222,3,74.0
"""2020""","""37.51""",426,6,71.0
"""2021""","""33.51""",32,2,16.0
"""2021""","""33.52""",702,9,78.0
"""2021""","""37.51""",448,7,64.0
"""2022""","""33.52""",1102,8,137.75
"""2022""","""37.51""",354,6,59.0
